### PAIRSQC
Pairs QC is run on every pairs file which have a source_experiments value. 
User submitted files will not have this field if they are not connected to the experimens through pseudo-runs.
(Not sure if that actually fills the field though)
We have old processed files that don't have this field which are skipped. 

#### Running on user files
If you want to run this process on user submitted pairs files, please patch the source_experiments field with uuid of the experiment(s), that way enzyme etc can be determined.

#### To-Do

<input type="checkbox" checked> add fruit fly info
<input type="checkbox" checked> add accession for chicken chrsize file.
<input type="checkbox"> move to foursight

...

In [2]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.wfr import *

# get key from keypairs.json
my_env = 'data'
my_key = get_key('koray_data')

# switch to run or not
add_wfr = True

# url for hic exps
pairs_url = "/search/?file_format.file_format=pairs"+ \
            "&status=pre-release&status=released&status=released+to+project&status=uploaded&type=FileProcessed" + \
            "&quality_metric.uuid=No+value&limit=all&source_experiments!=No value"
pairs_files = ff_utils.search_metadata(pairs_url , key=my_key)
print(len(pairs_files))
# for i in pairs_files:
#     print (i['genome_assembly'])

0


In [4]:
# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately


counter = 0
completed = 0
okay = 0
non = 0
running = 0
for a_pairs in pairs_files:

    # get experiment info
    exp_acc = a_pairs.get('source_experiments')
    if not exp_acc:
        print(a_pairs['accession'], 'does not have source experiments, skipping')
        continue

    #get enzyme and chrsize files
    nz_num, chrsize = extract_nz_file(exp_acc[0], my_key)    
    if not nz_num:
        print(nz_num, chrsize)
        print(a_pairs['accession'], 'does not have RE/chrsize, skipping')
        continue
                         
    # get report
    report = get_wfr_out(a_pairs['accession'], 'pairsqc-single', my_key, ['0.2.5', '0.2.6'], md_qc=True)
    
    attributions = get_attribution(a_pairs)
    # if report does not provide a complete or running run
    if report['status'].startswith('no'):
        non += 1
        if add_wfr:
            parameters = {"enzyme": nz_num,
                          "sample_name": a_pairs['accession']} 
            
            # if mouse, parameters get an additional parameter
            if chrsize == '4DNFI3UBJ3HZ':
                parameters['max_distance']= 8.2
            # if fly, parameters get an additional parameter
            if chrsize == '4DNFIBEEN92C':
                parameters['max_distance']= 7.5
            # if chicken, parameters get an additional parameter
            if chrsize == '4DNFIQFZW4DX':
                parameters['max_distance']= 8.2
            
            run_guide = {'wf_name': 'pairsqc-single',
                         'wf_uuid': 'b8c533e0-f8c0-4510-b4a1-ac35158e27c3',
                         'parameters': parameters,
                         'wfr_meta': attributions}

            inp_f = {'input_pairs':a_pairs['accession'], 'chromsizes':chrsize} 
            print(a_pairs['accession'], 'starting the run')
            run_missing_wfr(run_guide, inp_f, a_pairs['accession'], my_key, my_env)
        else:
            print(a_pairs['accession'], 'needs a run')
    
    # if there is a completed or still running run
    elif report['status'] == 'running':
        running += 1
        print(a_pairs['accession'], 'still running')
    elif report['status'] == 'complete':
        okay += 1
    # if there is something funky
    else:
        print('report status is not covered, please check')
        break

print("okay, not okay, running")    
print(okay, non, running)

4DNFI9FQOK8L still running
okay, not okay, running
0 0 1


In [ ]:
# Release pairs qc

# last run on  April 11 2019

last_run_date = "20190411"
pairs_url = '/search/?file_format.file_format=pairs&status=released&status=released%20to%20project&type=FileProcessed&limit=all'
if last_run_date:
    pairs_url += '&date_created.from=' + last_run_date
    
pairs_files = ff_utils.search_metadata(pairs_url , key=my_key)
pairs_files = [i for i in pairs_files if i.get('quality_metric')]
print(len(pairs_files))

patch = True
need_patch = 0
need_patch_run = 0
it = 0
for pairs in pairs_files:
    it += 1
    pairs_status = pairs['status']
    assert pairs_status in ['released', 'released to project']
        
    qc_uuid = pairs['quality_metric']['uuid']
    qc_status = ff_utils.get_metadata(qc_uuid,key=my_key)['status']
    if qc_status != pairs_status:
        print(pairs['accession'], it, 'qc_status_udate')
        need_patch += 1
        patch_data = {"status": pairs_status}
        if patch:
            ff_utils.patch_metadata(patch_data, obj_id=qc_uuid ,key=my_key)
        else:
            print('qc needs update')
    
    # output wfrs
    qc_wfr = [i for i in pairs['workflow_run_inputs'] if i['display_title'].startswith('pairsqc-single')]
    if not qc_wfr:
        print(it, pairs['accession'], 'does not have wfr')
        continue
    
    if len(qc_wfr) > 1:
        print(pairs['accession'], it, 'multiple wfrs!!!')
        continue
    
    wfr_status = ff_utils.get_metadata(qc_wfr[0]['uuid'], my_key)['status']
    if wfr_status != pairs_status:
        print(pairs['accession'], it, 'wfr_status_udate')
        need_patch_run += 1
        patch_data = {"status": pairs_status}
        if patch:
            ff_utils.patch_metadata(patch_data, obj_id=qc_wfr[0]['uuid'] ,key=my_key)
        else:
            print('wfr needs update')
        
    
print(need_patch)
print(need_patch_run)

In [ ]:
print(it)